<h1><font color=darkred size=6><center> $\fbox{Classification Multimodale - Rakuten}$ </center></h1>
<font color=dimgrey size=3><center> $Bryan, \space Rizlène, \space Romain\space et \space Steeve$ </center></h1>

# Table des matières

## [Importation des librairies](#Imp)

## [1. Analyse Descriptive](#Analyse)
- [1.1. Problématique](#Problématique)
- [1.2. Présentation des données](#PrezDonnees)
    - [1.2.1. Le jeu de données textuel](#Donnees_Textuelles)
    - [1.2.2. Le jeu de données visuel](#Donnees_Visuelles)
- [1.3. Visualisation des données](#VizDonnees)
    - [1.3.1. Analyse préliminaire](#Analyse_Preli)

    
## [Bibliographies](#Bibli)

<h2><font size=6> Importation des librairies<a name="Imp"></a></h2>

In [3]:
import os
import csv
import pandas as pd
import warnings
import matplotlib.pyplot as plt
import matplotlib.image as mpimg

warnings.filterwarnings('ignore')
pd.options.display.max_rows = 500
pd.options.display.max_columns = None

<h2><font size=6> 1. Analyse Descriptive <a name="Analyse"></a></h2>

<h3><font size=5> 1.1. Problématique <a name="Problématique"></a></h3>

Dans le cadre de ce projet, nous nous intéressons à la classification multimodale à grande échelle des produits sur la plateforme Rakuten France, en utilisant à la fois des données textuelles et visuelles. La problématique centrale réside dans la nécessité de catégoriser efficacement un vaste catalogue de produits, qui comprend des titres, des descriptions et des images, en codes de type de produit spécifiques. Cette tâche est d'une importance cruciale pour améliorer la recherche personnalisée et les recommandations, tout en faisant face aux défis liés à la diversité des produits, à la qualité variable des données et à la distribution déséquilibrée des classes. L'objectif est de développer un modèle de classification capable de traiter ces informations hétérogènes afin de réduire la duplication et d'optimiser l'expérience utilisateur sur la plateforme.

<h3><font size=5> 1.2. Présentation des données <a name="PrezDonnees"></a></h3>

Pour ce projet, nous disposons d'un ensemble de données comprenant environ 99 000 annonces de produits au format CSV, qui se divise en deux ensembles : un ensemble d'entraînement et un ensemble de test. Voici les détails des données :

<h4><font size=4> 1.2.1. Le jeu de données textuel <a name="Donnees_Textuelles"></a></h4>

**Ensemble d'Entraînement (X_train.csv et Y_train.csv) :**
   - **X_train.csv** : Contient les informations d'entrée pour l'entraînement, avec les colonnes suivantes :
     - **ID** : Un identifiant entier pour chaque produit.
     - **designation** : Le titre du produit, un texte court résumant le produit.
     - **description** : Un texte plus détaillé décrivant le produit (peut contenir des valeurs manquantes).
     - **productid** : Un identifiant unique pour le produit.
     - **imageid** : Un identifiant unique pour l'image associée au produit.
   - **Y_train.csv** : Contient les sorties d'entraînement, avec la colonne :
     - **prdtypecode** : Le code de type de produit correspondant à chaque ID dans X_train.csv.  

**Ensemble de Test (X_test.csv) :**
   - **X_test.csv** : Contient les informations d'entrée pour le test, avec la même structure que X_train.csv. Les participants doivent prédire les codes de type de produit pour cet ensemble.

**Exemple de Données :**
- **X_train.csv** :


In [4]:
X_train = pd.read_csv("../data/X_train_update.csv", sep=",").drop(["Unnamed: 0"], axis=1)
X_train.head(2)

,designation,description,productid,imageid
0,Olivia: Personalisiertes Notizbuch / 150 Seite...,NaN,3804725264,1263597046
1,Journal Des Arts (Le) N° 133 Du 28/09/2001 - L...,NaN,436067568,1008141237


- **Y_train.csv** :

In [5]:
Y_train = pd.read_csv("../data/Y_train.csv", sep=",").drop(["Unnamed: 0"], axis=1)
Y_train.head(2)

,prdtypecode
0,10
1,2280


Cette structure de données permet d'associer chaque produit à ses caractéristiques textuelles et visuelles, facilitant ainsi le développement d'un modèle de classification efficace pour prédire les codes de type de produit.

<h4><font size=4> 1.2.2. Le jeu de données visuel
    <a name="Donnees_Visuelles"></a></h4>

**Images :**
   - Un fichier compressé (images.zip) contenant toutes les images associées aux produits. Après décompression, deux sous-dossiers sont créés :
     - **image_train** : Contient les images des produits de l'ensemble d'entraînement.
     - **image_test** : Contient les images des produits de l'ensemble de test.
   - Les noms de fichiers d'images suivent le format : `image_imageid_product_productid.jpg`.

<h3><font size=5> 1.3. Visualisation des données <a name="VizDonnees"></a></h3>

Nous allons maintenant faire un peu de visualisation sur les différentes bases de données pour déceler de potentiels pattern.

<h4><font size=4> 1.3.1. Analyse préliminaire
    <a name="Analyse_Preli"></a></h4>

Dans cette étape de notre projet, nous allons procéder à une analyse préliminaire des bases de données. Nous examinerons tout d'abord le nombre de valeurs manquantes présentes dans les ensembles de données. Ensuite, nous vérifierons si tous les éléments du jeu de données textuels sont correctement associés à une image correspondante dans le jeu de données visuels.

In [6]:
X_train.info()

# Calculer le pourcentage de données manquantes
pourcentage_manquant = X_train["description"].isnull().mean() * 100
print(f"\nPourcentage de données manquantes : {pourcentage_manquant:.2f}%")

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 84916 entries, 0 to 84915
Data columns (total 4 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   designation  84916 non-null  object
 1   description  55116 non-null  object
 2   productid    84916 non-null  int64 
 3   imageid      84916 non-null  int64 
dtypes: int64(2), object(2)
memory usage: 2.6+ MB

Pourcentage de données manquantes : 35.09%


On remarque qu'il y a environ 35% de produits qui n'ont pas de description. Ce pourcentage étant élevé, on verra par la suite comment on le prendra en compte :
- Allons-nous supprimer ces données manquantes ? 
- Ces descriptions manquantes sont-elles associés à des types de produits particuliers ?

Checkons maintenant si tous nos produits sont associés à une image.

In [8]:
# Lister les fichiers d'images dans le dossier
fichiers_images = os.listdir("../data/images/image_train")

# Construire les noms d'images attendus
X_train['nom_image'] = 'image_' + X_train['imageid'].astype(str) + '_product_' + X_train['productid'].astype(str) + '.jpg'

# Vérifier combien de noms d'images sont présents dans le dossier
X_train['image_existe'] = X_train['nom_image'].isin(fichiers_images)

# Compter le nombre de lignes associées à une image
nombre_lignes_associees = X_train['image_existe'].sum()

print(f"Nombre de lignes totales : {len(X_train)}")
print(f'Nombre de lignes associées à une image : {nombre_lignes_associees}')

Nombre de lignes totales : 84916
Nombre de lignes associées à une image : 84916


On remarque qu'effectivement, tout le jeu de données textuel est bien associé à une image du jeu de données visuel.